<a href="https://colab.research.google.com/github/oneoftwo/KAIST_CH453_AI_chemistry/blob/main/assignments/assignment_1/assignment_1_2_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# DO NOT CHANGE!
!rm -r CH*
!rm -r assignment*
!rm -r practice*
!git clone https://github.com/oneoftwo/KAIST_CH453_AI_chemistry/
!mv ./KAIST_CH453_AI_chemistry/assignments/assignment_1/ ./
!rm -r KAIST_CH453*
!pip install rdkit
import rdkit
print(rdkit.__version__)
!ls

rm: cannot remove 'CH*': No such file or directory
rm: cannot remove 'assignment*': No such file or directory
rm: cannot remove 'practice*': No such file or directory
Cloning into 'KAIST_CH453_AI_chemistry'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 25 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 1.27 MiB | 3.71 MiB/s, done.
Resolving deltas: 100% (4/4), done.
2024.03.5
assignment_1  sample_data


# Dataset

In [ ]:
import pandas as pd

import torch
from torch.utils.data import Dataset
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import RDLogger
RDLogger.DisableLog("rdApp.*")


class BBBPDataset(Dataset):
    def __init__(self, smiles_list, target_list, fp_dim):
        assert len(smiles_list) == len(target_list)
        self.smiles_list = smiles_list
        self.target_list = target_list
        self.fp_dim = fp_dim

    def __len__(self):
        return len(self.smiles_list)

    def __getitem__(self, idx):
        smiles = self.smiles_list[idx]
        mol = Chem.MolFromSmiles(smiles)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=self.fp_dim)
        fp = torch.Tensor(fp)
        return fp, self.target_list[idx]


In [ ]:
import torch
from torch import nn


class Model(nn.Module):
    def __init__(self, fp_dim, hid_dim, n_layer, dropout):
        super().__init__()
        # TODO

    def forward(self, fp):
        # TODO


In [ ]:
from torch.nn import functional as F
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm


def process(model, data_loader, optimizer=None):
    # TODO
    return loss, acc, aucroc


In [ ]:
import numpy as np
import random
import torch
from torch import optim
from torch.utils.data import DataLoader


# arguments
fp_dim = 256
hid_dim = 256
dropout = 0.2
n_layer = 2

# do not change
bs = 32

# pre-process dataset
train_fn = "./assignment_1/train.csv"
valid_fn = "./assignment_1/valid.csv"
test_fn = "./assignment_1/test.csv"
df = pd.read_csv(train_fn)
train_smiles_list, train_target_list = df["smiles"].tolist(), df["Class"].tolist()
df = pd.read_csv(valid_fn)
valid_smiles_list, valid_target_list = df["smiles"].tolist(), df["Class"].tolist()
df = pd.read_csv(test_fn)
test_smiles_list, test_target_list = df["smiles"].tolist(), df["Class"].tolist()

# construct dataset
train_set = BBBPDataset(train_smiles_list, train_target_list, fp_dim=fp_dim)
valid_set = BBBPDataset(valid_smiles_list, valid_target_list, fp_dim=fp_dim)
test_set = BBBPDataset(test_smiles_list, test_target_list, fp_dim=fp_dim)

# make data loader
train_loader = DataLoader(train_set, batch_size=bs, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_set, batch_size=bs, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=bs, shuffle=True, num_workers=2)

# initialize model
model = Model(fp_dim, hid_dim, n_layer, dropout)
model.cuda()
print(sum(p.numel() for p in model.parameters() if p.requires_grad))
optimizer = optim.Adam(model.parameters(), lr=1e-3)
print(model, "\n")

# train
best_aucroc = 0.0
for _ in range(3):
    model.train()
    train_result = process(model, train_loader, optimizer)
    model.eval()
    with torch.no_grad():
        valid_result = process(model, valid_loader)

    print(_)
    print(train_result)
    print(valid_result)
    if valid_result[1] > best_aucroc:
        best_aucroc = valid_result[1]
    print(best_aucroc)

test_aucroc = process(model, test_loader)
print("test")
print(test_aucroc)


66049
Model(
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=1, bias=True)
  )
) 

0
(0.6146574522319593, 0.6972704714640199, 0.7625271912284941)
(0.5358940899372101, 0.7615894039735099, 0.8289803220035779)
0.7615894039735099
1
(0.48037265633281906, 0.7907361455748553, 0.8638406099624266)
(0.48543896675109866, 0.8211920529801324, 0.8466905187835421)
0.8211920529801324
2
(0.4259307494288997, 0.8064516129032258, 0.8908285908901143)
(0.45124844908714296, 0.8211920529801324, 0.8584973166368516)
0.8211920529801324
test
(0.49104424118995665, 0.7631578947368421, 0.8237344093910491)
